## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-04-05-51-25 +0000,bbc,F1 tycoon pleads guilty in rare Singapore corr...,https://www.bbc.com/news/articles/ceqylpw3zzdo...
1,2025-08-04-05-44-34 +0000,bbc,Boeing defence workers go on strike in new blo...,https://www.bbc.com/news/articles/c4gze2medkdo...
2,2025-08-04-05-27-47 +0000,bbc,Some working families 'cannot afford toilet pa...,https://www.bbc.com/news/articles/clyv77gl0zeo...
3,2025-08-04-05-21-29 +0000,bbc,Global issues cause islands' charities tough t...,https://www.bbc.com/news/articles/cgk36037jxvo...
4,2025-08-04-05-12-06 +0000,nyt,Long-Dormant Russian Volcano Erupts for the Fi...,https://www.nytimes.com/2025/08/03/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
81,trump,31
12,new,9
67,house,8
102,he,7
31,russian,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...,80
32,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...,76
9,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,72
25,2025-08-04-01-57-10 +0000,nypost,Diehard soccer fan kicked out of St. Louis sta...,https://nypost.com/2025/08/03/us-news/diehard-...,59
24,2025-08-04-02-03-23 +0000,nyt,Trump ‘wants his own people’ in the Bureau of ...,https://www.nytimes.com/live/2025/08/03/us/tru...,57


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,80,2025-08-03-14-22-11 +0000,nypost,Trump rips ‘sleazebag’ Charlamagne tha God aft...,https://nypost.com/2025/08/03/us-news/trump-ri...
32,45,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...
179,43,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...
155,43,2025-08-03-10-34-36 +0000,nypost,Ukrainian drone attack sparks massive fire at ...,https://nypost.com/2025/08/03/world-news/ukrai...
167,34,2025-08-03-09-30-00 +0000,wsj,"Like Zohran Mamdani in New York, the rise of M...",https://www.wsj.com/politics/elections/sociali...
55,34,2025-08-03-22-56-15 +0000,nypost,Boeing St. Louis defense workers set to strike...,https://nypost.com/2025/08/03/business/boeing-...
9,28,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...
79,27,2025-08-03-20-37-30 +0000,bbc,Manhunt intensifies for Montana bar shooting s...,https://www.bbc.com/news/articles/cy8j79yw8pqo...
72,27,2025-08-03-21-13-37 +0000,nyt,Third Child in Sailing Camp Dies After Barge H...,https://www.nytimes.com/2025/08/03/us/miami-be...
100,26,2025-08-03-18-17-00 +0000,cbc,Krasheninnikov volcano in Russia's Far East er...,https://www.cbc.ca/news/world/kamchatka-penins...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
